# DALI expressions and arithmetic operators

In this example, we will see how to use arithmetic operators in DALI Pipeline that alow for elementwise operations on tensors inside a pipeline. We will explain the type promotion rules and show examples of using currently supported arithmetic operators, namely `+`, `-`, `*`, `/` and `//`.

## Prepare the test pipeline

First, we will prepare helper code, so we can easily manipulate the types and values that will appear as tensors in DALI pipeline.

We use `from __future__ import division` to allow `/` and `//` as true division and floor division operators.
We will be using numpy as source for the custom provided data and we also need to import several things from DALI needed to create Pipeline and use ExternalSource Operator. 

In [1]:
from __future__ import division
import numpy as np
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops            
import nvidia.dali.types as types
from nvidia.dali.types import Constant

batch_size = 1

### Defining the data

As we are dealing with binary operators, we need two inputs. 
We will create a simple helper function that returns two numpy arrays of given numpy types with arbitrary selected values. It is to make the manipulation of types easy. In an actual scenario the data processed by DALI arithmetic operators would be tensors produced by other Operator containing some images, video sequences or other data.

Keep in mind that shapes of both inputs need to match as those will be elementwise operations. 

In [2]:
left_magic_values = [42, 8]
right_magic_values = [9, 2]

def get_data(left_type, right_type):
    return ([left_type(left_magic_values)], [right_type(right_magic_values)])

batch_size = 1

### Defining the pipeline

The next step is to define the Pipeline. We override `Pipeline.iter_setup`, a method called by the pipeline before every `Pipeline.run`. It is meant to feed the data into `ExternalSource()` operators indicated by `self.left` and `self.right`.
The data will be obtained from `get_data` function to which we pass the left and right types. 

Note, that we do not need to instantiate any additional operators, we can use regular Python arithmetic expression on the results of other operators in the `define_graph` step.

For convinience reason we will wrap the usage arithmetic operators in a lambda, so we can specify it when creating the pipeline. We will call that argument `operation`.

`define_graph` will return both our data inputs and the result of applying `operation` to them.

In [3]:
 class ArithmeticPipeline(Pipeline):                   
    def __init__(self, operation, left_type, right_type, batch_size, num_threads, device_id):
        super(ArithmeticPipeline, self).__init__(batch_size, num_threads, device_id, seed=12)
        self.left_source = ops.ExternalSource()
        self.right_source = ops.ExternalSource()
        self.operation = operation
        self.left_type = left_type
        self.right_type = right_type

    def define_graph(self):                                                                
        self.left = self.left_source()
        self.right = self.right_source()
        return self.left, self.right, self.operation(self.left, self.right)

    def iter_setup(self):
        (l, r) = get_data(self.left_type, self.right_type)
        self.feed_input(self.left, l)
        self.feed_input(self.right, r)

## Experiment with the arithmetic operators

### Instantiating the Pipeline

We create instances of the `ArithmeticPipeline` with different type combinations. 

Type promotions for binary operators are described below. They apply to `+`, `-`, `*` and `//`. The `/` always returns a float32 for integer inputs, and applies the rules below when at least one of the inputs is a floating point number.

| Operand Type | Operand Type | Result Type | Additional Conditions |
|:------------:|:------------:|:-----------:| --------------------- |
| T      | T      | T                |                        |
| floatX | T      | floatX           | where T is not a float |
| floatX | floatY | float(max(X, Y)) |                        |
| intX   | intY   | int(max(X, Y))   |                        |
| uintX  | uintY  | uint(max(X, Y))  |                        |
| intX   | uintY  | int2Y            | if X <= Y              |
| intX   | uintY  | intX             | if X > Y               |

### Using the Pipeline

Let's create a Pipeline that adds two tensors of type `uint8`, run it and see the results.

In [11]:
def build_and_run(pipe, op_name):
    pipe.build()                                                        
    pipe_out = pipe.run()
    l = pipe_out[0].as_array()
    r = pipe_out[1].as_array()
    out = pipe_out[2].as_array()
    print("{} {} {} = {}; \t\t\t with types {} {} {} -> {}".format(l, op_name, r, out, l.dtype, op_name, r.dtype, out.dtype))
    
pipe = ArithmeticPipeline((lambda x, y: x + y), np.uint8, np.uint8, batch_size = batch_size, num_threads = 2, device_id = 0)
build_and_run(pipe, "+")

[[42  8]] + [[9 2]] = [[51 10]]; 			 with types uint8 + uint8 -> uint8


Let's see how all of the operators behave with different type combinations by generalizing the example above.
You can use the `np_types` in the loops to see all possible type combinatons. To reduce the oputput we limit ourselves to only few of them.

In [13]:
arithmetic_operations = [((lambda x, y: x + y) , "+"), ((lambda x, y: x - y) , "-"),
                         ((lambda x, y: x * y) , "*"), ((lambda x, y: x / y) , "/"),
                         ((lambda x, y: x // y) , "//")]

np_types = [np.int8, np.int16, np.int32, np.int64, 
            np.uint8, np.uint16, np.uint32, np.uint64,
            np.float32, np.float64]

for (op, op_name) in arithmetic_operations:
    for left_type in [np.uint8]:
        for right_type in [np.uint8, np.int32, np.float32]:
            pipe = ArithmeticPipeline(op, left_type, right_type, batch_size=batch_size, num_threads=2, device_id = 0)
            build_and_run(pipe, op_name)

[[42  8]] + [[9 2]] = [[51 10]]; 			 with types uint8 + uint8 -> uint8
[[42  8]] + [[9 2]] = [[51 10]]; 			 with types uint8 + int32 -> int32
[[42  8]] + [[9. 2.]] = [[51. 10.]]; 			 with types uint8 + float32 -> float32
[[42  8]] - [[9 2]] = [[33  6]]; 			 with types uint8 - uint8 -> uint8
[[42  8]] - [[9 2]] = [[33  6]]; 			 with types uint8 - int32 -> int32
[[42  8]] - [[9. 2.]] = [[33.  6.]]; 			 with types uint8 - float32 -> float32
[[42  8]] * [[9 2]] = [[122  16]]; 			 with types uint8 * uint8 -> uint8
[[42  8]] * [[9 2]] = [[378  16]]; 			 with types uint8 * int32 -> int32
[[42  8]] * [[9. 2.]] = [[378.  16.]]; 			 with types uint8 * float32 -> float32
[[42  8]] / [[9 2]] = [[4.6666665 4.       ]]; 			 with types uint8 / uint8 -> float32
[[42  8]] / [[9 2]] = [[4.6666665 4.       ]]; 			 with types uint8 / int32 -> float32
[[42  8]] / [[9. 2.]] = [[4.6666665 4.       ]]; 			 with types uint8 / float32 -> float32
[[42  8]] // [[9 2]] = [[4 4]]; 			 with types uint8 // uint8 -> u

### Using Constants

Instead of operating only on Tensor data, DALI expressions can also work with constants. Those can be either values of Python `int` and `float` types used directly, or those values wrapped in `nvidia.dali.types.Constant`.

*Note: Currently all values of integral constants are passed to DALI as int32 and all values of floating point constants are passed to DALI as float32.*

The Python `int` values will be treated as `int32` and the `float` as `float32` in regard to type promotions.

The DALI `Constant` can be used to indicate other types. It accepts `DALIDataType` enum values as second argument and has convinience member functions like `.uint8()` or `.float32()` that can be used for conversions.